In [1]:
import ciropt as co
import cvxpy as cp

import PEPit
import PEPit.functions as pep_func
import ciropt.function as co_func
import sympy as sp

In [2]:
L_smooth = 1.
mu = 0.1 #0

solver = "ipopt"
# solver = "ipopt_qcqp"
# solver = "ipopt_qcqp_matrix" 

# Ciropt problem

In [3]:
problem = co.accelerated_gradient_circuit_tuneRLC(mu, L_smooth)
problem.obj = problem.eta + problem.rho

bounds = {"C":{"ub": 10, "lb":0.5},
          "R":{"ub": 10, "lb":0.5},
          "L":{"ub": 10, "lb":0.5}}

res, model, sp_exp = problem.solve(solver=solver, bounds=bounds, verbose=False, debug=True)

dim_G=6, dim_F=4
Ipopt total # of variables = 374
Actual # of variables = 49


In [4]:
res

{'C': 10.000000098593643,
 'L': 10.00000004985383,
 'R': 0.49999999005637524,
 'alpha': 0.25662344259673525,
 'beta': -0.17464682307232068,
 'eta': 2.78344092932915,
 'h': 10.644399118460987,
 'invC': 0.09999999901406355,
 'invL': 0.09999999950146175,
 'rho': 5.832952935283037}

# PEP verification

In [5]:
problem = co.accelerated_gradient_circuit_tuneRLC(mu, L_smooth, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=8.157456932167406e-10
